<a href="https://colab.research.google.com/github/HDuy197/ds102_lab5/blob/main/ds102_lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bài 1

In [1]:
!pip install py_vncorenlp

In [2]:
!mkdir -p /content/vncorenlp

import py_vncorenlp

py_vncorenlp.download_model(save_dir='/content/vncorenlp')
model_vncorenlp =py_vncorenlp.VnCoreNLP(annotators=["wseg"],
                            save_dir="/content/vncorenlp",
                            max_heap_size='-Xmx2g')

VnCoreNLP model folder /content/vncorenlp already exists! Please load VnCoreNLP from this folder!


In [3]:
import gdown
import os
import json

data_dir = '/content/uit_vsfc_data'
os.makedirs(data_dir, exist_ok=True)

file_ids = {
    "train.json": "1eJdEStCQBf3GtbOIQfIR2iS-Wb3nlfJX",
    "dev.json": "1UWbzHFeUPu4AuofOEy0Fo0SFotVZHzUe",
    "test.json": "1g4imdabNbswIu91K6zKFGalkjDBBqaQq"
}

for filename, file_id in file_ids.items():
    output_path = os.path.join(data_dir, filename)
    gdown.download(id=file_id, output=output_path, quiet=False)

def load_data(filename):
    with open(os.path.join(data_dir, filename), 'r', encoding='utf-8') as f:
        return json.load(f)

train_data = load_data('train.json')
dev_data = load_data('dev.json')
test_data = load_data('test.json')

Downloading...
From: https://drive.google.com/uc?id=1eJdEStCQBf3GtbOIQfIR2iS-Wb3nlfJX
To: /content/uit_vsfc_data/train.json
100%|██████████| 2.02M/2.02M [00:00<00:00, 91.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1UWbzHFeUPu4AuofOEy0Fo0SFotVZHzUe
To: /content/uit_vsfc_data/dev.json
100%|██████████| 274k/274k [00:00<00:00, 46.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1g4imdabNbswIu91K6zKFGalkjDBBqaQq
To: /content/uit_vsfc_data/test.json
100%|██████████| 559k/559k [00:00<00:00, 38.8MB/s]


In [6]:
  def segment_text_field(data_list, vncorenlp_model):
      for idx, item in enumerate(data_list):
          raw_text = item.get('sentence', '')
          segmented_words = []
          try:
              if not raw_text.strip():
                  raise ValueError("Câu rỗng hoặc chỉ chứa khoảng trắng.")
              annotated = vncorenlp_model.annotate_text(raw_text)
              if not isinstance(annotated, dict) or 0 not in annotated:
                  raise ValueError(f"Kết quả không chứa khóa 0: {annotated}")
              for word_dict in annotated[0]:
                  segmented_words.append(word_dict['wordForm'])
              item['segmented_text'] = " ".join(segmented_words)
          except Exception as e:
              print(f"[LỖI tại index {idx}] {e}\n→ Câu: '{raw_text}'")
              item['segmented_text'] = raw_text
      return data_list

  train_data = segment_text_field(train_data, model_vncorenlp)
  dev_data = segment_text_field(dev_data, model_vncorenlp)
  test_data = segment_text_field(test_data, model_vncorenlp)

  for i in range(5):
      print(f"Câu gốc   : {train_data[i]['sentence']}")
      print(f"Đã tách từ: {train_data[i]['segmented_text']}\n")

Câu gốc   : slide giáo trình đầy đủ .
Đã tách từ: slide giáo_trình đầy_đủ .

Câu gốc   : nhiệt tình giảng dạy , gần gũi với sinh viên .
Đã tách từ: nhiệt_tình giảng_dạy , gần_gũi với sinh_viên .

Câu gốc   : đi học đầy đủ full điểm chuyên cần .
Đã tách từ: đi học đầy_đủ full điểm chuyên_cần .

Câu gốc   : chưa áp dụng công nghệ thông tin và các thiết bị hỗ trợ cho việc giảng dạy .
Đã tách từ: chưa áp_dụng công_nghệ_thông_tin và các thiết_bị hỗ_trợ cho việc giảng_dạy .

Câu gốc   : thầy giảng bài hay , có nhiều bài tập ví dụ ngay trên lớp .
Đã tách từ: thầy giảng bài hay , có nhiều bài_tập ví_dụ ngay trên lớp .



## Bài 2

In [7]:
!pip install transformers

In [9]:
  import torch
  import numpy as np
  import json
  from transformers import AutoModel, AutoTokenizer

  train_texts = [item['segmented_text'] for item in train_data if item.get('segmented_text')]
  dev_texts   = [item['segmented_text'] for item in dev_data if item.get('segmented_text')]
  test_texts  = [item['segmented_text'] for item in test_data if item.get('segmented_text')]

  phobert = AutoModel.from_pretrained("vinai/phobert-base-v2")
  tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

  def get_phobert_embeddings(sentences, tokenizer, model, max_length=256, batch_size=64):
      embeddings = []
      model.eval()
      for i in range(0, len(sentences), batch_size):
          batch_texts = sentences[i:i + batch_size]
          if not batch_texts or all(not t.strip() for t in batch_texts):
              continue
          inputs = tokenizer(batch_texts, return_tensors='pt', max_length=max_length,
                            padding=True, truncation=True)
          with torch.no_grad():
              outputs = model(**inputs)
              cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
              embeddings.append(cls_embeddings)
      return np.concatenate(embeddings, axis=0)

  train_embeddings = get_phobert_embeddings(train_texts, tokenizer, phobert)
  dev_embeddings   = get_phobert_embeddings(dev_texts, tokenizer, phobert)
  test_embeddings  = get_phobert_embeddings(test_texts, tokenizer, phobert)

  np.save('/content/uit_vsfc_data/train_embeddings.npy', train_embeddings)
  np.save('/content/uit_vsfc_data/dev_embeddings.npy', dev_embeddings)
  np.save('/content/uit_vsfc_data/test_embeddings.npy', test_embeddings)

  print(f"Kích thước:")
  print("Train:", train_embeddings.shape)
  print("Dev:  ", dev_embeddings.shape)
  print("Test: ", test_embeddings.shape)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Kích thước:
Train: (11426, 768)
Dev:   (1583, 768)
Test:  (3166, 768)


## Bài 3

In [1]:
  import numpy as np
  import json
  from itertools import permutations

  with open('/content/uit_vsfc_data/train.json', 'r', encoding='utf-8') as f:
      train_data = json.load(f)
  with open('/content/uit_vsfc_data/test.json', 'r', encoding='utf-8') as f:
      test_data = json.load(f)

  train_embeddings = np.load('/content/uit_vsfc_data/train_embeddings.npy')
  test_embeddings = np.load('/content/uit_vsfc_data/test_embeddings.npy')

  np.random.seed(42)
  n_dev = int(0.2 * len(train_data))
  indices = np.random.permutation(len(train_data))
  dev_indices = indices[:n_dev]
  train_indices = indices[n_dev:]

  dev_data = [train_data[i] for i in dev_indices]
  train_data_new = [train_data[i] for i in train_indices]
  dev_embeddings = train_embeddings[dev_indices]
  train_embeddings_new = train_embeddings[train_indices]

  def standardize_data(data):
      mean = np.mean(data, axis=0)
      std = np.std(data, axis=0) + 1e-6
      return (data - mean) / std

  train_embeddings_new = standardize_data(train_embeddings_new)
  dev_embeddings = standardize_data(dev_embeddings)
  test_embeddings = standardize_data(test_embeddings)

  def extract_labels(data):
      label_map = {'positive': 0, 'neutral': 1, 'negative': 2}
      return np.array([label_map[item['sentiment'].lower()] for item in data])

  train_labels_new = extract_labels(train_data_new)
  dev_labels = extract_labels(dev_data)
  test_labels = extract_labels(test_data)

  def gmm_em(data, n_clusters=3, n_iterations=100, tol=1e-4):
      n_samples, n_features = data.shape
      np.random.seed(42)
      means = data[np.random.choice(n_samples, n_clusters, replace=False)]
      covariances = np.array([np.cov(data.T) + np.eye(n_features) * 1e-6 for _ in range(n_clusters)])
      weights = np.ones(n_clusters) / n_clusters
      responsibilities = np.zeros((n_samples, n_clusters))
      log_2pi = n_features * np.log(2 * np.pi)

      for _ in range(n_iterations):
          for k in range(n_clusters):
              diff = data - means[k]
              inv_cov = np.linalg.inv(covariances[k])
              log_det = np.log(np.linalg.det(covariances[k]) + 1e-6)
              exponent = -0.5 * np.sum(diff @ inv_cov * diff, axis=1)
              log_prob = np.log(weights[k] + 1e-6) - 0.5 * (log_2pi + log_det) + exponent
              responsibilities[:, k] = log_prob

          max_log_prob = np.max(responsibilities, axis=1, keepdims=True)
          responsibilities = np.exp(responsibilities - max_log_prob)
          responsibilities /= np.sum(responsibilities, axis=1, keepdims=True)

          Nk = np.sum(responsibilities, axis=0)
          means_new = (responsibilities.T @ data) / Nk[:, None]

          for k in range(n_clusters):
              diff = data - means_new[k]
              covariances[k] = (responsibilities[:, k][:, None] * diff).T @ diff / Nk[k]
              covariances[k] += np.eye(n_features) * 1e-6

          weights_new = Nk / n_samples
          if np.all(np.abs(means_new - means) < tol):
              break

          means, weights = means_new, weights_new

      return means, covariances, weights

  def predict_labels(data, means, covariances, weights):
      n_samples, n_features = data.shape
      n_clusters = len(means)
      responsibilities = np.zeros((n_samples, n_clusters))
      log_2pi = n_features * np.log(2 * np.pi)

      for k in range(n_clusters):
          diff = data - means[k]
          inv_cov = np.linalg.inv(covariances[k])
          log_det = np.log(np.linalg.det(covariances[k]) + 1e-6)
          exponent = -0.5 * np.sum(diff @ inv_cov * diff, axis=1)
          log_prob = np.log(weights[k] + 1e-6) - 0.5 * (log_2pi + log_det) + exponent
          responsibilities[:, k] = log_prob

      max_log_prob = np.max(responsibilities, axis=1, keepdims=True)
      responsibilities = np.exp(responsibilities - max_log_prob)
      return np.argmax(responsibilities, axis=1)

  def compute_metrics(true_labels, pred_labels):
      n_classes = len(set(true_labels))
      best_f1 = 0
      best_perm = None

      for perm in permutations(range(n_classes)):
          mapped = np.array([perm[p] for p in pred_labels])
          precision, recall, f1 = [], [], []

          for c in range(n_classes):
              tp = np.sum((mapped == c) & (true_labels == c))
              fp = np.sum((mapped == c) & (true_labels != c))
              fn = np.sum((mapped != c) & (true_labels == c))

              p = tp / (tp + fp + 1e-8)
              r = tp / (tp + fn + 1e-8)
              f = 2 * p * r / (p + r + 1e-8)

              precision.append(p)
              recall.append(r)
              f1.append(f)

          if np.mean(f1) > best_f1:
              best_f1 = np.mean(f1)
              best_result = (precision, recall, f1, perm)

      return best_result

  means, covariances, weights = gmm_em(train_embeddings_new, n_clusters=3)

  dev_preds = predict_labels(dev_embeddings, means, covariances, weights)
  precision_dev, recall_dev, f1_dev, best_perm_dev = compute_metrics(dev_labels, dev_preds)
  print("Precision (Dev):", precision_dev)
  print("Recall (Dev):", recall_dev)
  print("F1 Score (Dev):", f1_dev)
  print(f"Mean F1 (Dev): {np.mean(f1_dev):.4f}")

  test_preds = predict_labels(test_embeddings, means, covariances, weights)
  precision_test, recall_test, f1_test, best_perm_test = compute_metrics(test_labels, test_preds)
  print("Precision (Test):", precision_test)
  print("Recall (Test):", recall_test)
  print("F1 Score (Test):", f1_test)
  print(f"Mean F1 (Test): {np.mean(f1_test):.4f}")

Precision (Dev): [np.float64(0.49890590809409674), np.float64(0.0), np.float64(0.0)]
Recall (Dev): [np.float64(0.9999999999912281), np.float64(0.0), np.float64(0.0)]
F1 Score (Dev): [np.float64(0.6656934262118516), np.float64(0.0), np.float64(0.0)]
Mean F1 (Dev): 0.2219
Precision (Test): [np.float64(0.5022109917861586), np.float64(0.0), np.float64(0.0)]
Recall (Test): [np.float64(0.9999999999937108), np.float64(0.0), np.float64(0.0)]
F1 Score (Test): [np.float64(0.6686290956303261), np.float64(0.0), np.float64(0.0)]
Mean F1 (Test): 0.2229
